In [ ]:
from datasets import load_dataset
huggingface_dataset = load_dataset("raquiba/Sarcasm_News_Headline",data_dir="")
huggingface_dataset = huggingface_dataset["train"]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/repocard.py:105: UserWarning: Repo card metadata block was not found. Setting CardData to empty.
  warnings.warn("Repo card metadata block was not found. Setting CardData to empty.")


Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [ ]:
pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.8 MB/s eta 0:00:00


In [ ]:
import pandas as pd

In [ ]:
df = pd.DataFrame(huggingface_dataset)

In [ ]:
df.head()

is_sarcastic                                           headline  \
0             1  thirtysomething scientists unveil doomsday clo...   
1             0  dem rep. totally nails why congress is falling...   
2             0  eat your veggies: 9 deliciously different recipes   
3             1  inclement weather prevents liar from getting t...   
4             1  mother comes pretty close to using word 'strea...   

                                        article_link  
0  https://www.theonion.com/thirtysomething-scien...  
1  https://www.huffingtonpost.com/entry/donna-edw...  
2  https://www.huffingtonpost.com/entry/eat-your-...  
3  https://local.theonion.com/inclement-weather-p...  
4  https://www.theonion.com/mother-comes-pretty-c...

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import nltk
from sklearn.preprocessing import LabelBinarizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from wordcloud import WordCloud,STOPWORDS
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize,sent_tokenize
from bs4 import BeautifulSoup
import re,string,unicodedata
from keras.preprocessing import text, sequence
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
from sklearn.model_selection import train_test_split
from string import punctuation
import keras
from keras.models import Sequential
from keras.layers import Dense,Embedding,LSTM,Dropout,Bidirectional,GRU
import tensorflow as tf

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
stop = set(stopwords.words('english'))
punctuation = list(string.punctuation)
stop.update(punctuation)

In [ ]:
def strip_html(text):
    soup = BeautifulSoup(text, "html.parser")
    return soup.get_text()

#Removing the square brackets
def remove_between_square_brackets(text):
    return re.sub('\[[^]]*\]', '', text)
# Removing URL's
def remove_between_square_brackets(text):
    return re.sub(r'http\S+', '', text)
#Removing the stopwords from text
def remove_stopwords(text):
    final_text = []
    for i in text.split():
        if i.strip().lower() not in stop:
            final_text.append(i.strip())
    return " ".join(final_text)
#Removing the noisy text
def denoise_text(text):
    text = strip_html(text)
    text = remove_between_square_brackets(text)
    text = remove_stopwords(text)
    return text
#Apply function on review column
df['headline']=df['headline'].apply(denoise_text)

<ipython-input-12-ba69b7b9ea42>:2: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, "html.parser")


In [ ]:
words = []
for i in df.headline.values:
    words.append(i.split())
words[:5]

[['thirtysomething',
  'scientists',
  'unveil',
  'doomsday',
  'clock',
  'hair',
  'loss'],
 ['dem',
  'rep.',
  'totally',
  'nails',
  'congress',
  'falling',
  'short',
  'gender,',
  'racial',
  'equality'],
 ['eat', 'veggies:', '9', 'deliciously', 'different', 'recipes'],
 ['inclement', 'weather', 'prevents', 'liar', 'getting', 'work'],
 ['mother',
  'comes',
  'pretty',
  'close',
  'using',
  'word',
  "'streaming'",
  'correctly']]

In [ ]:
import gensim
#Dimension of vectors we are generating
EMBEDDING_DIM = 200

#Creating Word Vectors by Word2Vec Method (takes time...)
w2v_model = gensim.models.Word2Vec(sentences = words , vector_size=EMBEDDING_DIM , window = 5 , min_count = 1)

In [ ]:
tokenizer = text.Tokenizer(num_words=35000)
tokenizer.fit_on_texts(words)
tokenized_train = tokenizer.texts_to_sequences(words)
x = sequence.pad_sequences(tokenized_train, maxlen = 20)

In [ ]:
vocab_size = len(tokenizer.word_index) + 1

In [ ]:
def get_weight_matrix(model, vocab):
    # total vocabulary size plus 0 for unknown words
    vocab_size = len(vocab) + 1
    # define weight matrix dimensions with all 0
    weight_matrix = np.zeros((vocab_size, EMBEDDING_DIM))
    # step vocab, store vectors using the Tokenizer's integer mapping
    for word, i in vocab.items():
        weight_matrix[i] = model[word]
    return weight_matrix

In [ ]:
#Getting embedding vectors from word2vec and using them as weights of non-trainable Keras embedding layer
def get_weight_matrix(model, vocab):
    # Total vocabulary size plus 0 for unknown words
    vocab_size = len(vocab) + 1
    # Define matrix for embedding layer with all zeros
    weight_matrix = np.zeros((vocab_size, EMBEDDING_DIM))
    # Step vocab, store vectors using the Tokenizer's integer mapping
    for word, i in vocab.items():
        try:
            weight_matrix[i] = model.wv[word]
        except KeyError:
            # Word not found in the Word2Vec model vocabulary, leave as zeros
            pass
    return weight_matrix

embedding_vectors = get_weight_matrix(w2v_model, tokenizer.word_index)


In [ ]:
model = Sequential()
#Non-trainable embeddidng layer
model.add(Embedding(vocab_size, output_dim=EMBEDDING_DIM, weights=[embedding_vectors], input_length=20, trainable=True))
#LSTM
model.add(Bidirectional(LSTM(units=128 , recurrent_dropout = 0.3 , dropout = 0.3,return_sequences = True)))
model.add(Bidirectional(GRU(units=32 , recurrent_dropout = 0.1 , dropout = 0.1)))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer=keras.optimizers.Adam(lr = 0.01), loss='binary_crossentropy', metrics=['acc'])

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, df.is_sarcastic , test_size = 0.3 , random_state = 0)

In [ ]:
history = model.fit(x_train, y_train, batch_size = 128 , validation_data = (x_test,y_test) , epochs = 3)

Epoch 1/3
157/157 [==============================] - 106s 595ms/step - loss: 0.5375 - acc: 0.7126 - val_loss: 0.4208 - val_acc: 0.8039
Epoch 2/3
157/157 [==============================] - 100s 637ms/step - loss: 0.2122 - acc: 0.9143 - val_loss: 0.4781 - val_acc: 0.8042
Epoch 3/3
157/157 [==============================] - 91s 579ms/step - loss: 0.0669 - acc: 0.9761 - val_loss: 0.7270 - val_acc: 0.7936


In [ ]:
print("Accuracy of the model on Testing Data is - " , model.evaluate(x_test,y_test)[1]*100)

269/269 [==============================] - 8s 28ms/step - loss: 0.7270 - acc: 0.7936
Accuracy of the model on Testing Data is -  79.36175465583801


In [ ]:
# Define a function to predict the class of a sentence
def predict_sarcasm(sentence):
    # Preprocess the input sentence
    cleaned_sentence = denoise_text(sentence)
    # Tokenize the preprocessed sentence
    tokenized_sentence = tokenizer.texts_to_sequences([cleaned_sentence])
    # Pad the sequence
    padded_sentence = sequence.pad_sequences(tokenized_sentence, maxlen=20)
    # Predict the class
    prediction = model.predict(padded_sentence)[0][0]
    if prediction >= 0.5:
        return "Sarcasm"
    else:
        return "Not Sarcasm"

# Test the function with an example sentence
example_sentence = "That was a hilarious joke"
prediction = predict_sarcasm(example_sentence)
print("Prediction for the sentence '{}' is: {}".format(example_sentence, prediction))

1/1 [==============================] - 2s 2s/step
Prediction for the sentence 'That was a hilarious joke' is: Sarcasm


In [ ]:
from datasets import load_dataset
huggingface_dataset = load_dataset("nutorbit/news-headline-gen",data_dir="")
huggingface_dataset = huggingface_dataset["train"]

Generating train split:   0%|          | 0/21157 [00:00<?, ? examples/s]

Generating dev split:   0%|          | 0/2365 [00:00<?, ? examples/s]

In [ ]:
df = pd.DataFrame(huggingface_dataset)

In [ ]:
df.head()

headline  \
0   30K Walmart Part-Timers to Lose Health Insurance   
1     Dax Shepard: Wedding to Kristen Bell Cost $142   
2                            Nancy Reagan Dead at 94   
3  American Airlines Faces $7M Fine for Safety Vi...   
4   $222K Raised for Kids of Mom Dismembered on Date   

                                                news  
0  (Oct 7, 2014  12:40 PM CDT) As of Jan. 1, Walm...  
1  (Oct 29, 2013  8:15 AM CDT) Dax Shepard and Kr...  
2  (Mar 6, 2016  10:50 AM) Nancy Reagan, the help...  
3  (Aug 15, 2008  5:11 AM CDT) American Airlines ...  
4  (Apr 18, 2016  1:02 PM CDT) Ingrid Lyne, the S...